In [1]:
import sys
import os
sys.path.insert(0, os.getcwd()+'/src/')
sys.path.insert(0, '/home/mcarenso/code/stage-2023-multiscale-extremes/scripts/src/')
from myImports import *
%matplotlib inline
%load_ext autoreload

In [5]:
stringSST = "300" ##295, 300 or 305
n_days = 20

#TOOCAN segmentation masks (les labels des objets MCS, dans la grille originale x,y,t)
file_seg='/bdd/MT_WORKSPACE/MCS/RCE/SAM/TOOCAN/TOOCAN_v2022_04/irtb/TOOCAN_2.07_SAM_RCE_large'+stringSST+'_2D_irtb.nc'

# TOOCAN objects (list d'objets MCS, leur labels et leur caractéristiques)
file_tracking='/bdd/MT_WORKSPACE/MCS/RCE/SAM/TOOCAN/TOOCAN_v2022_04/irtb/FileTracking/TOOCAN-SAM_RCE_large'+stringSST+'_2D_irtb.dat.gz'

sam_dir_path = "/bdd/MT_WORKSPACE/REMY/RCEMIP/SAM/300K/"

# Let's Big Data

In [6]:
## GPT dask fever

import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import dask.distributed as ddistributed
import dask.delayed as delayed

In [7]:
# Create a Dask client
client = ddistributed.Client()

# Open native precip datasets
ds1 = xr.open_dataset(sam_dir_path+"rcemip_large_2048x128x74_3km_12s_"+stringSST+"K_64.2Dcom_1.nc", chunks = {'time' :48, 'x' : 32, 'y' : 32})
ds2 = xr.open_dataset(sam_dir_path+"rcemip_large_2048x128x74_3km_12s_"+stringSST+"K_64.2Dcom_2.nc", chunks = {'time' :48, 'x' : 32, 'y' : 32})

# Combine datasets
ds = xr.concat([ds1, ds2], dim='time')

# Rename dimensions
ds["x"] = ((ds["x"])/3e3).astype(int)
ds["y"] = (ds["y"]/3e3).astype(int) 
ds["time"] = np.round(((ds["time"]-75)*48)).astype(int) 

In [10]:
ds

,Array,Chunk
Bytes,1.17 GiB,192.00 kiB
Shape,"(1201, 128, 2048)","(48, 32, 32)"
Count,13314 Tasks,6656 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,1.17 GiB,192.00 kiB
Shape,"(1201, 128, 2048)","(48, 32, 32)"
Count,13314 Tasks,6656 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [8]:
ds_SAM_shape= ds["Prec"].shape
#ds_SAM_Prec = ds_SAM["Prec"].stack(flat=('time', 'y', 'x')).to_numpy()
#ds_SAM_Prec.reshape(ds_SAM_shape)[226, 76, 1747] ## to retrieve the PRec in original shape or ds_SAM["Prec"].to_numpy()
dist_SAM_Prec = cs.Distribution(name="SAM Precipitation", bintype = "invlogQ", nd = 6, fill_last_decade=True)
dist_SAM_Prec.computeDistribution(sample = ds["Prec"].stack(flat=('time', 'y', 'x')))
dist_SAM_Prec.storeSamplePoints(sample = ds["Prec"].stack(flat=('time', 'y', 'x')), sizemax = None)
SAM_Prec_bin_loc = dist_SAM_Prec.bin_locations

/home/mcarenso/.conda/envs/PyLMD/lib/python3.8/site-packages/numpy/lib/histograms.py:905: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges


AttributeError: 'DataArray' object has no attribute 'flatten'

In [13]:
temp = "300"
file_seg='/bdd/MT_WORKSPACE/MCS/RCE/SAM/TOOCAN/TOOCAN_v2022_04/irtb/TOOCAN_2.07_SAM_RCE_large'+temp+'_2D_irtb.nc'   ## TODO PPass arguemnt temp correctly
n_days = 25
label_mask = xr.open_dataarray(file_seg) # .isel(time=slice(48*n_days)) ## TODO Pass arguemnt n_days correctly
mask = ~np.isnan(label_mask.values)

da = ds["Prec"].where(mask)

In [14]:
da

,Array,Chunk
Bytes,1.17 GiB,192.00 kiB
Shape,"(1201, 128, 2048)","(48, 32, 32)"
Count,33283 Tasks,6656 Chunks
Type,float32,numpy.ndarray
